In [1]:
import numpy as np
import pandas as pd

#### Monte carlo simulations

In [84]:
def call_payoff(S:float, K:float, r:float, q:float, T:float) -> float:
    return max(S*np.exp(-q*T)-K*np.exp(-r*T), 0)

def put_payoff(S:float, K:float, r:float, q:float, T:float) -> float:
    return max(K*np.exp(-r*T)-S*np.exp(-q*T), 0)

def monte_carlo(S0:float, K:float, sigma:float, rf:float, q:float, t:float, NOofsteps:int, NOofPaths:int) -> pd.DataFrame:
        """Geometric Brownian Motion"""
        r = rf - q
        np.random.seed(42)
        random = np.random.normal(0, 1, size = (NOofsteps, NOofPaths))
        paths = np.zeros(shape=(NOofsteps, NOofPaths))
        paths[0,:]=S0
        dt=t/NOofsteps
        for i in range(NOofPaths):
            for j in range(1, NOofsteps):
                paths[j,i] = paths[j-1,i]*np.exp((r-(sigma**2)/2)*dt + sigma*random[j-1,i]*np.sqrt(dt))
        simulations = [f"simulation_{i}" for i in range(NOofPaths)]
        
        m_c = pd.DataFrame(data=paths, columns=simulations)
        
        paths = m_c
        
        S_t = m_c.iloc[-1:,]
        
        S_t_call = [call(S, K, rf, q, t) for S in S_t.values[0]]
        
        S_t_put = [put(S, K,  rf, q, t) for S in S_t.values[0]]
        
        return paths, np.mean(S_t_call), np.mean(S_t_put)

def monte_carlo_vectorized(S0:float, K:float, sigma:float, rf:float, q:float, t:float, NOofsteps:int, NOofPaths:int) -> pd.DataFrame:
    
    r = rf - q
    np.random.seed(42)
    random = np.random.normal(0, 1, size = (NOofsteps, NOofPaths))
    dt=t/NOofsteps
    exp_ = np.exp((rf - (sigma**2)/2)*dt + sigma*random*np.sqrt(dt))
    exp_cumprod = np.cumprod(exp_, axis=0)
    paths = exp_cumprod*S0
    paths[0,:] = S0
    
    S_t = paths[-1, :]

    S_t_call = [call_payoff(S, K, rf, q, t) for S in S_t]
        
    S_t_put = [put_payoff(S, K,  rf, q, t) for S in S_t]
    
    
    return paths, S_t_call, S_t_put
    

####  Pathwise Greeks

In [102]:
def Pathwise_delta(opt_type, S0, K, path, r, t):
    temp_1 = path[-1, :] > K
    call_delta = np.exp(-r*t)*np.mean(path[-1:,]/S0*temp_1)
    if opt_type.lower() == "call":
        return call_delta
    elif opt_type.lower() == "put":
        return -(1-call_delta)

def Pathwise_vega(S0, K, path, sigma, r, t):
    temp_1 = path[-1, :] > K
    temp_2 = 1.0/sigma * S[:, -1]
    call_vega = np.exp(-r*t)*np.mean(path[-1:,]/S0*temp_1)